test the api with a simple call

In [27]:
import finnhub

# Setup client
finnhub_client = finnhub.Client(api_key="crc2tvpr01qtpc71itugcrc2tvpr01qtpc71itv0")

# Stock candles
#res = finnhub_client.stock_candles('AAPL', 'D', 1590988249, 1591852249)
#print(res)

print(finnhub_client.symbol_lookup('BINANCE:BTCUSDT'))


{'count': 1, 'result': [{'description': 'Binance BTC/USDT', 'displaySymbol': 'BTC/USDT', 'symbol': 'BINANCE:BTCUSDT', 'type': 'Crypto'}]}


create a socket and recieve (limiting to 25 message) and process them based on the task:
Use the finnhub.io websockets API to retrieve in real-time data the bitcoin price from the Binance exchange. You'll need to following symbol: BINANCE:BTCUSDT. The payload is json, so use the json module in Python to transform the message to dict.
1. Your class should have the optionto output in the console all relevant trades in the following format:
2. Implement a data structure which calculates time averages. For each one minute period (e.g. 10:00-10:01, 10:01-10:02, etc.) calculate the volume-weighted average priceof trades made during this minute. Keep in mind that there may be late-arriving data, and the messages you receive are not guaranteed to be ordered.

In [40]:
#https://pypi.org/project/websocket_client/
import json
import websocket
from datetime import datetime

dict_averages = {}

# connect to my custom message processing
def custom_formating(ws, message):
    my_dict = json.loads(message)
    
    for tr in my_dict['data']:    
        # convert second to a timestamp
        dt = datetime.fromtimestamp(float(tr['t']) / 1000.0 )
        
        # visualize recieved input
        print(dt.strftime('%Y-%m-%d %H:%M:%S'),'price:' + str(tr['p']).format('f2'), 'volume:' + str(tr['v']))
   
        # accumulate data
        # use a dict having key - our period (convert date to minutes)
        dt_key = datetime.fromtimestamp(float(tr['t']) / 1000.0 ).strftime('%Y-%m-%d %H:%M')

        # since we use the key to be the date, this will avoid the unordered or late messages
        if (dt_key in dict_averages):
            dict_averages[str(dt_key)][0] += tr['p'] * tr['v']
            dict_averages[str(dt_key)][1] += tr['v']
        else:
            # create first element
            dict_averages[str(dt_key)] = [tr['p']*tr['v'], tr['v']]

    
    # limit the messages to 50 for processing
    ws.messages_count+=1
    if ws.messages_count>20:
        ws.close()
    
# default processing
def on_message(ws, message):
    print(message)
    
def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    #ws.send('{"type":"subscribe","symbol":"AAPL"}')
    #ws.send('{"type":"subscribe","symbol":"AMZN"}')
    ws.send('{"type":"subscribe","symbol":"BINANCE:BTCUSDT"}')
    #ws.send('{"type":"subscribe","symbol":"IC MARKETS:1"}')

if __name__ == "__main__":
    websocket.enableTrace(False)
    ws = websocket.WebSocketApp("wss://ws.finnhub.io?token=crc2tvpr01qtpc71itugcrc2tvpr01qtpc71itv0",
                              on_message = custom_formating,
                              on_error = on_error,
                              on_close = on_close)
    ws.messages_count = 0
    ws.on_open = on_open
    ws.run_forever()
    
    ws.close()

    # print(dict_averages)
    print("volume-weighted average prices:")
    for key, values in dict_averages.items():
        print(key, values[0]/values[1])
    
    

Websocket connected


2024-09-04 16:40:14 price:56429.99 volume:0.0007
2024-09-04 16:40:15 price:56430 volume:0.05263
2024-09-04 16:40:15 price:56430 volume:0.00039
2024-09-04 16:40:15 price:56430 volume:0.00288
2024-09-04 16:40:15 price:56430 volume:0.08804
2024-09-04 16:40:15 price:56430 volume:0.08804
2024-09-04 16:40:15 price:56430 volume:0.01761
2024-09-04 16:40:15 price:56430 volume:0.01761
2024-09-04 16:40:15 price:56430 volume:0.17607
2024-09-04 16:40:15 price:56430 volume:0.1
2024-09-04 16:40:15 price:56430 volume:0.00088
2024-09-04 16:40:15 price:56430 volume:0.00129
2024-09-04 16:40:15 price:56430.04 volume:0.00015
2024-09-04 16:40:15 price:56430.55 volume:0.0001
2024-09-04 16:40:15 price:56430.55 volume:0.0001
2024-09-04 16:40:15 price:56430.55 volume:0.0001
2024-09-04 16:40:15 price:56430.55 volume:0.0001
2024-09-04 16:40:15 price:56431.02 volume:0.00044
2024-09-04 16:40:15 price:56431.09 volume:0.0001
2024-09-04 16:40:15 price:56431.09 volume:0.0001
2024-09-04 16:40:15 price:56431.09 volume:0.

error from callback <function on_close at 0x000001B2943D7920>: on_close() takes 1 positional argument but 3 were given


2024-09-04 16:40:16 price:56433.8 volume:0.08826
2024-09-04 16:40:16 price:56441.09 volume:0.0001
2024-09-04 16:40:16 price:56431.1 volume:0.07088
2024-09-04 16:40:16 price:56431.1 volume:0.0024
2024-09-04 16:40:16 price:56431.1 volume:0.00022
2024-09-04 16:40:16 price:56431.1 volume:0.00149
2024-09-04 16:40:16 price:56441.09 volume:0.0001
2024-09-04 16:40:16 price:56441.09 volume:0.0001
2024-09-04 16:40:16 price:56441.1 volume:0.07087
2024-09-04 16:40:16 price:56431.45 volume:0.0001
2024-09-04 16:40:16 price:56431.45 volume:0.0001
2024-09-04 16:40:16 price:56431.45 volume:0.0001
2024-09-04 16:40:16 price:56431.45 volume:0.0001
2024-09-04 16:40:16 price:56431.45 volume:0.0001
2024-09-04 16:40:16 price:56431.45 volume:0.0001
2024-09-04 16:40:16 price:56442 volume:0.01
2024-09-04 16:40:16 price:56431.45 volume:0.0001
2024-09-04 16:40:16 price:56431.45 volume:0.0001
2024-09-04 16:40:16 price:56442 volume:0.01
2024-09-04 16:40:16 price:56442.99 volume:0.0001
2024-09-04 16:40:16 price:56431

pip install websocket-client --user
pip install websocket --user